# geom #2

In [ ]:
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
L = 1;   E = 200e9;   I = 1e-6;   q = 10000.0;   c = q / (E * I)
geom2 = dde.geometry.Interval(0, L)

In [ ]:
def ode2(x, y):
    dy_xx = dde.grad.hessian(y, x)
    dy_xxxx = dde.grad.hessian(dy_xx, x)
    return dy_xxxx + c

def solution(x):
    return (-q / (24 * E * I)) * (x**4 - 2 * L * x**3 + L**2 * x**2)

In [ ]:
def boundary_l(x, on_boundary):
    return on_boundary and dde.utils.isclose(x[0], 0)

def boundary_r(x, on_boundary):
    return on_boundary and dde.utils.isclose(x[0], L)

bcD_l = dde.icbc.DirichletBC(geom2, lambda x: 0, boundary_l)
bcN_l = dde.icbc.NeumannBC(geom2, lambda x: 0, boundary_l)

bcD_r = dde.icbc.DirichletBC(geom2, lambda x: 0, boundary_r)
bcN_r = dde.icbc.NeumannBC(geom2, lambda x: 0, boundary_r)

bcs2 = [bcD_l, bcN_l, bcD_r, bcN_r]

In [ ]:
data2 = dde.data.PDE(geom2, ode2, bcs2, num_domain=100, num_boundary=50, solution=solution)

In [ ]:
net2 = dde.nn.FNN([1, 50, 50, 50, 1], "swish", "Glorot uniform")
model2 = dde.Model(data2, net2)
model2.compile("adam", lr=1e-4, metrics=['l2 relative error'], loss_weights=[1, 10, 10, 10, 10])
model2.train(iterations=30000, display_every=1000)
model2.compile("L-BFGS", loss_weights=[1, 10, 10, 10, 10])
model2.train()

In [ ]:
X2 = np.linspace(0, L, 200)[:, None]
y_pred2 = model2.predict(X2)
w_analytical2 = solution(X2)


plt.figure(figsize=(10, 6))
plt.plot(X2, y_pred2, label="PINN Prediction", color="red", linestyle="--")
plt.plot(X2, w_analytical2, label="Exact Solution", color="blue")
plt.xlabel("x")
plt.ylabel("w(x)")
plt.title("PINN vs Analytical: Deflection of Fully Restrained Beam Under Uniform Load")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# import deepxde as dde
# import numpy as np
# import tensorflow as tf

In [ ]:
# # Physical constants
# L = 1.0  # Length of the beam
# E = 210e9  # Young's modulus (Pa)
# I = 1e-6  # Moment of inertia (m^4)
# P = 1000.0  # Point load (N)
# epsilon = L / 100.0  # Spread of the Gaussian approximation
# geom = dde.geometry.Interval(0.0, L)

In [ ]:
# def delta(x, center=L/2):
#     """Gaussian approximation of Dirac delta centered at L/2."""
#     return 1.0 / (epsilon * np.sqrt(np.pi)) * np.exp(-((x - center) ** 2) / (epsilon ** 2))

In [ ]:
# def pde(x, w):
#     # First derivative dw/dx
#     w_x = dde.grad.jacobian(w, x)
#     # Second derivative d2w/dx2
#     w_xx = dde.grad.hessian(w, x)
#     # Third derivative d3w/dx3
#     w_xxx = dde.grad.jacobian(w_xx, x)
#     # Fourth derivative d4w/dx4
#     w_xxxx = dde.grad.jacobian(w_xxx, x)
#     # Gaussian delta term (TensorFlow)
#     delta_tf = tf.exp(-((x - L/2) ** 2) / (epsilon ** 2)) / (epsilon * tf.sqrt(np.pi))
#     return E * I * w_xxxx + P * delta_tf

In [ ]:
# bc_w0 = dde.DirichletBC(geom, lambda x: 0.0, lambda x, on_boundary: on_boundary and np.isclose(x[0], 0.0))
# bc_dw0 = dde.NeumannBC(geom, lambda x: 0.0, lambda x, on_boundary: on_boundary and np.isclose(x[0], 0.0))
# # w(L) = 0 and w'(L) = 0 at x=L
# bc_wL = dde.DirichletBC(geom, lambda x: 0.0, lambda x, on_boundary: on_boundary and np.isclose(x[0], L))
# bc_dwL = dde.NeumannBC(geom, lambda x: 0.0, lambda x, on_boundary: on_boundary and np.isclose(x[0], L))

In [ ]:
# data = dde.data.PDE(geom, pde, [bc_w0, bc_dw0, bc_wL, bc_dwL], num_domain=200, num_boundary=100)

In [ ]:
# layer_size = [1] + [50] * 3 + [1]
# activation = 'tanh'
# initializer = 'Glorot uniform'
# net = dde.maps.FNN(layer_size, activation, initializer)

# model = dde.Model(data, net)
# model.compile('adam', lr=1e-3)
# history = model.train(epochs=5000)
# model.compile('L-BFGS')
# history = model.train()

In [ ]:
# def analytic_w(x):
#     x = x[:, 0]
#     w = np.zeros_like(x)
#     C = P / (48 * E * I)
#     for i, xi in enumerate(x):
#         if xi <= L / 2:
#             w[i] = C * (3 * L * xi**2 - 4 * xi**3)
#         else:
#             w[i] = C * (3 * L * xi**2 - 4 * xi**3 + L**3 - 6 * L**2 * (xi - L / 2))
#     return w.reshape(-1, 1)

# X = np.linspace(0, L, 100)[:, None]
# w_pred = model.predict(X)
# w_true = analytic_w(X)

# print("Predicted max deflection:", np.min(w_pred))
# print("Analytical max deflection:", np.min(w_true))